# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** I have chosen an Encoder-Decoder architecture. The Encoder module that was provided to us in the workspace is a resnet50 model that was pretrained on ImageNet. This module enables us to extract features from images. The final layer of this module has been replaced with a fully-connected Linear layer to transform image features into embeddings.

The Decoder module is based on an LSTM architecture. There is an embedding layer to transform captions into embeddings. Next comes the unidirectional LSTM layer. Lastly, I have a dropout layer for avoiding overfitting and a fully-connected linear layer to transform the output into vocabulary matrix.

For my first attempt in training the model, I used `batch_size = 64`, `vocab_threshold = 5`, `embed_size = 300`, `hidden_size = 128`, `num_epochs = 3`. However, during inference my model was predicting the same sentence for all images. I resolved that issue by slightly modifying my model's sample method after following some advise from the Knowledge/Q&A platform. The results were still not satisfying. Examining the output captions, it felt like my model was underfitting a bit. So, I decided to increase `hidden_size` and `num_epochs`. Also, I decreased `vocab_thresold` to allow a larger number of vocabulary keys. I also wanted to increase the `batch_size` to test whether the computing environment can handle a larger `batch_size`.


Variable values - final selections:
- **batch_size = 128** - This gave about 3000 steps per batch which seems reasonable to me. Also, the computing environment had enough memory to handle this `batch_size`.
- **vocab_threshold = 3** - I wanted to allow more vocabulary keys as I was training my model for longer and with larger `hidden_size` in my second attempt.
- **embed_size = 300** - Based on my previous readings on Natural Language Processing methods and on the discussion in one of the lessons in this course, I felt confident in choosing 300 as the final value for this project.
- **hidden_size = 256** - Using a size of 128 did not seem to overfit my model. However, I did not complete the optional validation part of this assignment. So, I am not sure about this. But, judging from the model predictions, I definitely felt that I got much better results when using a size of 256.
- **num_epochs = 5** - Training for 5 epochs took roughly 13 hours. Since the model weights are saved every epoch, I decided to run the training for 5 epochs from noon to midnight. If lower a number of epochs gives better results, I can always use those saved weights.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The provided defaults of the `transform_train` transform are reasonable. The Resizing (followed by RandomCrop) gives an image size that is pretty standard for deep learning today. Conversion to Tensor is an essential preprocessing step. The Normalization values are also standard for many computer vision models (including our pretrained resnet50). RandomCrop and RandomHorizontalFlip provide some data augmentation. No further augmentations seems necessary given the relatively large size of the dataset.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I chose to make all the weights in the Decoder and only the weights in the embedding layer of the Encoder trainable. By freezing the pretrained weights in the earlier layers of the Encoder and only training our custom embedding layer, we are essentially performing transfer learning and fine-tuning our image features by only learning the weights in the custom embedding layer. For the LSTM Decoder, we are not using any pretrained weights. Hence, all the weights in the Decoder have to be trained.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** In this second project, I could not do as much experimentation as I did in the Facial Keypoint Detection (FKD). This is due to limited GPU hours in the workspace and each experimentation run in this project taking much longer to complete. In the FKD project, I found that the model trains much faster (i.e. faster convergence) when using Adam optimizer compared to Stochastic Gradient Descent optimizer. Also, from my previous experiences with Adam optimizer, I only had to tune the learing rate. Adam optimizer's defualt parameters worked well for me in most cases. So, it seemed like an ideal choice given the limited GPU hours in the workspace.

In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size (try 64, 128)
vocab_threshold = 3        # minimum word count threshold (try 5, 3)
vocab_from_file = True    # if True, load existing vocab file
embed_size = 300           # dimensionality of image and word embeddings
hidden_size = 256          # number of features in hidden state of the RNN decoder (try 128, 256)
num_epochs = 5             # number of training epochs (try 3, 5)
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 355/414113 [00:00<01:56, 3548.07it/s]

  0%|          | 788/414113 [00:00<01:50, 3750.38it/s]

Done (t=0.89s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 1234/414113 [00:00<01:44, 3936.47it/s]

  0%|          | 1682/414113 [00:00<01:41, 4083.07it/s]

  1%|          | 2112/414113 [00:00<01:39, 4145.47it/s]

  1%|          | 2548/414113 [00:00<01:37, 4206.83it/s]

  1%|          | 2985/414113 [00:00<01:36, 4254.31it/s]

  1%|          | 3410/414113 [00:00<01:36, 4251.83it/s]

  1%|          | 3853/414113 [00:00<01:35, 4301.42it/s]

  1%|          | 4293/414113 [00:01<01:34, 4329.29it/s]

  1%|          | 4737/414113 [00:01<01:33, 4361.51it/s]

  1%|▏         | 5186/414113 [00:01<01:32, 4398.43it/s]

  1%|▏         | 5626/414113 [00:01<01:32, 4396.26it/s]

  1%|▏         | 6071/414113 [00:01<01:32, 4410.82it/s]

  2%|▏         | 6510/414113 [00:01<01:33, 4359.25it/s]

  2%|▏         | 6945/414113 [00:01<01:33, 4356.18it/s]

  2%|▏         | 7380/414113 [00:01<01:35, 4249.19it/s]

  2%|▏         | 7820/414113 [00:01<01:34, 4292.25it/s]

  2%|▏         | 8253/414113 [00:01<01:34, 4301.32it/s]

  2%|▏         | 8694/414113 

 28%|██▊       | 115044/414113 [00:26<01:07, 4450.25it/s]

 28%|██▊       | 115490/414113 [00:26<01:07, 4433.51it/s]

 28%|██▊       | 115934/414113 [00:26<01:08, 4381.45it/s]

 28%|██▊       | 116382/414113 [00:26<01:07, 4407.76it/s]

 28%|██▊       | 116823/414113 [00:26<01:07, 4378.77it/s]

 28%|██▊       | 117262/414113 [00:26<01:08, 4352.62it/s]

 28%|██▊       | 117716/414113 [00:26<01:07, 4405.79it/s]

 29%|██▊       | 118157/414113 [00:27<01:07, 4390.10it/s]

 29%|██▊       | 118602/414113 [00:27<01:07, 4407.81it/s]

 29%|██▊       | 119044/414113 [00:27<01:06, 4410.77it/s]

 29%|██▉       | 119496/414113 [00:27<01:06, 4440.40it/s]

 29%|██▉       | 119941/414113 [00:27<01:06, 4441.01it/s]

 29%|██▉       | 120391/414113 [00:27<01:05, 4456.25it/s]

 29%|██▉       | 120861/414113 [00:27<01:04, 4524.51it/s]

 29%|██▉       | 121319/414113 [00:27<01:04, 4540.47it/s]

 29%|██▉       | 121774/414113 [00:27<01:05, 4477.74it/s]

 30%|██▉       | 122223/414113 [00:27<01:05, 4458.33it/s

 55%|█████▍    | 227664/414113 [00:52<00:41, 4515.85it/s]

 55%|█████▌    | 228116/414113 [00:52<00:41, 4515.00it/s]

 55%|█████▌    | 228568/414113 [00:52<00:41, 4492.99it/s]

 55%|█████▌    | 229018/414113 [00:52<00:41, 4488.11it/s]

 55%|█████▌    | 229467/414113 [00:52<00:41, 4473.10it/s]

 56%|█████▌    | 229915/414113 [00:52<00:41, 4436.34it/s]

 56%|█████▌    | 230364/414113 [00:52<00:41, 4449.61it/s]

 56%|█████▌    | 230814/414113 [00:52<00:41, 4462.50it/s]

 56%|█████▌    | 231261/414113 [00:52<00:41, 4410.25it/s]

 56%|█████▌    | 231704/414113 [00:53<00:41, 4412.56it/s]

 56%|█████▌    | 232146/414113 [00:53<00:41, 4383.89it/s]

 56%|█████▌    | 232585/414113 [00:53<00:41, 4356.81it/s]

 56%|█████▋    | 233024/414113 [00:53<00:41, 4364.77it/s]

 56%|█████▋    | 233466/414113 [00:53<00:41, 4380.47it/s]

 56%|█████▋    | 233914/414113 [00:53<00:40, 4409.56it/s]

 57%|█████▋    | 234360/414113 [00:53<00:40, 4422.52it/s]

 57%|█████▋    | 234803/414113 [00:53<00:40, 4415.67it/s

 82%|████████▏ | 340101/414113 [01:17<00:17, 4333.61it/s]

 82%|████████▏ | 340542/414113 [01:18<00:16, 4354.49it/s]

 82%|████████▏ | 340978/414113 [01:18<00:16, 4352.89it/s]

 82%|████████▏ | 341418/414113 [01:18<00:16, 4365.10it/s]

 83%|████████▎ | 341867/414113 [01:18<00:16, 4398.92it/s]

 83%|████████▎ | 342307/414113 [01:18<00:16, 4377.70it/s]

 83%|████████▎ | 342756/414113 [01:18<00:16, 4409.47it/s]

 83%|████████▎ | 343198/414113 [01:18<00:16, 4361.70it/s]

 83%|████████▎ | 343635/414113 [01:18<00:16, 4357.61it/s]

 83%|████████▎ | 344071/414113 [01:18<00:16, 4354.06it/s]

 83%|████████▎ | 344512/414113 [01:18<00:15, 4370.46it/s]

 83%|████████▎ | 344950/414113 [01:19<00:15, 4370.96it/s]

 83%|████████▎ | 345388/414113 [01:19<00:15, 4361.91it/s]

 84%|████████▎ | 345829/414113 [01:19<00:15, 4375.37it/s]

 84%|████████▎ | 346267/414113 [01:19<00:15, 4370.57it/s]

 84%|████████▎ | 346705/414113 [01:19<00:15, 4360.14it/s]

 84%|████████▍ | 347145/414113 [01:19<00:15, 4368.52it/s

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d-batch-128-hidden-256-epochs-5.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d-batch-128-hidden-256-epochs-5.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/5], Step [100/3236], Loss: 4.0570, Perplexity: 57.80130
Epoch [1/5], Step [200/3236], Loss: 3.6369, Perplexity: 37.9741
Epoch [1/5], Step [300/3236], Loss: 3.5062, Perplexity: 33.3226
Epoch [1/5], Step [400/3236], Loss: 3.1912, Perplexity: 24.3185
Epoch [1/5], Step [500/3236], Loss: 3.4936, Perplexity: 32.9041
Epoch [1/5], Step [600/3236], Loss: 3.0461, Perplexity: 21.0330
Epoch [1/5], Step [700/3236], Loss: 3.1617, Perplexity: 23.61133
Epoch [1/5], Step [800/3236], Loss: 2.9947, Perplexity: 19.97960
Epoch [1/5], Step [900/3236], Loss: 2.8580, Perplexity: 17.4265
Epoch [1/5], Step [1000/3236], Loss: 2.8802, Perplexity: 17.8177
Epoch [1/5], Step [1100/3236], Loss: 3.0625, Perplexity: 21.3807
Epoch [1/5], Step [1200/3236], Loss: 2.9859, Perplexity: 19.8050
Epoch [1/5], Step [1300/3236], Loss: 2.7500, Perplexity: 15.6419
Epoch [1/5], Step [1400/3236], Loss: 2.6782, Perplexity: 14.5588
Epoch [1/5], Step [1500/3236], Loss: 2.7241, Perplexity: 15.2424
Epoch [1/5], Step [1600/3236], 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.